In [12]:
## Setup + data import

import pandas as pd
import matplotlib.pyplot as plt
from datetime import *

wikiData = pd.read_csv("../data/all_data/cleanWiki.csv")[["Article", "Language", "Month", "Count"]]
wikiData_reverted = pd.read_csv("../data/all_data/cleanWiki_reverted.csv")[["Article", "Language", "Month", "Count"]]
protections=pd.read_csv("../data/all_data/protections.csv")[['article', 'language', 'start', 'end']]
protections['start'] = pd.to_datetime(protections['start'], format='mixed')
protections['end'] = pd.to_datetime(protections['end'], format='mixed')


In [13]:
def convert_yyyy_mm(s: str):
    try:
        s += "-01"
        
        return datetime.strptime(s, "%Y-%m-%d")
    except:
        return None

wikiData['Date'] = wikiData['Month'].apply(convert_yyyy_mm)
wikiData_reverted['Date'] = wikiData_reverted['Month'].apply(convert_yyyy_mm)

In [14]:
import itertools

def insert_empty_intervals(df, oldest = datetime(2011,1,1)):

    articles = df['Article'].unique()
    langs = df['Language'].unique()
    dates = df['Date'].sort_values(ascending = True).unique()

    cartesian_product = list(itertools.product(articles, langs, dates))

    df_cartesian_product = pd.DataFrame(cartesian_product, columns=['Article', 'Language', 'Date'])

    result_df = pd.merge(df_cartesian_product, df, how = 'left', on = ['Article', 'Language', 'Date'])
    result_df['Count'] = result_df['Count'].fillna(0)
    result_df = result_df[result_df['Date'] > oldest]

    return result_df[['Article', 'Language', 'Date', 'Count']]

print(len(wikiData))
print(len(wikiData_reverted))

wikiData = insert_empty_intervals(wikiData)
wikiData_reverted = insert_empty_intervals(wikiData_reverted)

print(len(wikiData))
print(len(wikiData_reverted))

55782
6891
227911
96777


In [15]:
grouped = wikiData.groupby(['Article', 'Language'])['Count'].sum().reset_index()
grouped.rename(columns={'Count': 'Total_edits'}, inplace=True)
wikiData = pd.merge(wikiData, grouped, on=['Article', 'Language'])

grouped = wikiData_reverted.groupby(['Article', 'Language'])['Count'].sum().reset_index()
grouped.rename(columns={'Count': 'Total_edits'}, inplace=True)
wikiData_reverted = pd.merge(wikiData_reverted, grouped, on=['Article', 'Language'])

In [16]:
from scipy.stats import pearsonr
import matplotlib.pyplot as plt

def transform_count(article = None, lang = None, df = wikiData):

    if article is not None and lang is not None:
        filter_df = df[(df['Article'] == article) & (df['Language'] == lang)]
    else:
        filter_df = df

    median_value = filter_df['Count'].mean()
    return filter_df['Count'] / median_value * 100

def transform_counts(article1, lang1, article2, lang2, df = wikiData):

    filter_df_1 = df[(df['Article'] == article1) & (df['Language'] == lang1)]
    filter_df_2 = df[(df['Article'] == article2) & (df['Language'] == lang2)]

    filter_df_1 = filter_df_1[filter_df_1['Date'].isin(filter_df_2['Date'])]
    filter_df_2 = filter_df_2[filter_df_2['Date'].isin(filter_df_1['Date'])]

    series_1 = transform_count(df = filter_df_1)
    series_2 = transform_count(df = filter_df_2)
    
    return series_1, series_2

def correlation_edits(article1, lang1, article2, lang2, df = wikiData):

    series1, series2 = transform_counts(article1, lang1, article2, lang2, df)
    correlation, _ = pearsonr(series1, series2)
    
    return correlation, _

def plot_correlation(article1, lang1, article2, lang2, df = wikiData):
    
    series1, series2 = transform_counts(article1, lang1, article2, lang2, df)
    correlation, _ = pearsonr(series1, series2)
    print(pearsonr(series1, series2))

    plt.figure(figsize=(10, 6))
    plt.plot(df['Date'].unique(), series1, label= "{} , {}".format(article1, lang1) , color='blue')
    plt.plot(df['Date'].unique(), series2, label= "{} , {}".format(article2, lang2) , color='red')

    plt.xlabel('Date')
    plt.ylabel('Relative count of edits')
    plt.title('Pearson correlation: %.3f' % correlation)
    plt.legend()

    plt.show()
    

correlation_edits('Abraham_Lincoln', 'en', 'Abraham_Lincoln', 'de')[0]


0.04036625144109736

In [17]:
print(len(wikiData))

wikiData = wikiData[wikiData['Total_edits'] > 0]

print(len(wikiData))

227911
69004


In [58]:
wikiData.loc[wikiData['Total_edits'] > 200, ['Article', 'Language']].drop_duplicates()

,Article,Language
0,Benito_Mussolini,it
161,Benito_Mussolini,en
1449,Fall_of_the_Fascist_regime_in_Italy,it
1610,Fall_of_the_Fascist_regime_in_Italy,en
2898,Italian_Social_Republic,it
...,...,...
222759,Palestinian_cuisine,de
224369,Samih_al-Qasim,ar
225174,Mahmoud_Darwish,en
225818,Mahmoud_Darwish,ar


In [83]:
def calculate_correlations(df : pd.DataFrame = wikiData) -> pd.DataFrame:

    correlations = pd.DataFrame(columns=['Article1', 'Article2', 'Lang1', 'Lang2', 'Correlation', 'P_value'])
    
    df = df[df['Total_edits'] > 0]

    unique_pairs = list(df.drop_duplicates(subset=['Article', 'Language'])[['Article', 'Language']].to_records(index=False))

    for c in itertools.combinations(unique_pairs, 2):
        
        try:
 
            correlation = correlation_edits(c[0][0], c[0][1], c[1][0], c[1][1])
            row = pd.DataFrame({'Article1': [c[0][0]], 'Article2' : [c[1][0]], 'Lang1' : [c[0][1]], 'Lang2' : [c[1][1]], 'Correlation' : [correlation[0]], 'P_value' : [correlation[1]]})
            correlations=pd.concat([correlations, row])
        except:
            continue

    return correlations

def join_topics(topics_df : pd.DataFrame, df : pd.DataFrame) -> pd.DataFrame:
    
    df = pd.merge(df, topics_df, 'left', left_on = 'Article1', right_on = 'Article')[['Article1', 'Article2', 'Lang1', 'Lang2', 'Correlation', 'P_value', 'Topic']]
    df = pd.merge(df, topics_df, 'left', left_on = 'Article2', right_on = 'Article', suffixes = ['_1', '_2'])[['Article1', 'Article2', 'Lang1', 'Lang2', 'Correlation', 'P_value', 'Topic_1', 'Topic_2']]

    return df

def determine_group(row):
    
    return_value = ''
    if row['Article1'] == row['Article2'] and row['Lang1'] != row['Lang2']:
        return 'Same Article'
    
    if row['Topic_1'] == row['Topic_2']:
        return_value += 'Same Topic '
    
    if row['Lang1'] == row['Lang2']:
        return_value += 'Same Language'
        
    if return_value == '':
        return_value += 'Group 5'

    return return_value


def assign_groups(df):

    df['Group'] = df.apply(determine_group, axis=1)
    return df

def correlations_topics_groups(df: pd.DataFrame, topics_df: pd.DataFrame):

    df_correlations = calculate_correlations(df)
    print("Correlations calculated")

    df_topics = join_topics(topics_df, df_correlations)
    print("Topics assigned")

    df_groups = assign_groups(df_topics)
    print("Groups assigned")

    df_groups = df_groups.sort_values('Correlation', ascending = False)
    
    return df_groups

In [84]:
topics = {
    'Ukraine' : 
    ["Kyiv","Kievan_Rus'","Stepan_Bandera","Bohdan_Khmelnytsky","Cossacks","Ukrainian_language","Holodomor","Borscht","Symon_Petliura",
    "Ukrainian_People's_Republic","Mykhailo_Hrushevsky","Nikolai_Gogol","Taras_Shevchenko","Ukrainian_literature","Ivan_Franko","Ukrainian_Insurgent_Army","Organisation_of_Ukrainian_Nationalists","Pierogi",
    "Kolach_(bread)", "Paska_(bread)", "Pampushka", "Syrniki", "Rusyns", "Vyshyvanka", "Ukrainian_Soviet_Socialist_Republic", "Pereiaslav_Agreement", "West_Ukrainian_People's_Republic", 
    "Massacres_of_Poles_in_Volhynia_and_Eastern_Galicia", "Orange_Revolution", "Ukrainian_War_of_Independence", "Principality_of_Kiev", "Kyiv_Pechersk_Lavra","Golden_Gate,_Kyiv", "Bakhchysarai_Palace", 
    "Khreshchatyk", "Kamianets-Podilskyi_Castle",  "Saint_Sophia_Cathedral,_Kyiv", "Kobzar", "Hryhorii_Skovoroda", "Lesya_Ukrainka", "Rus'_people", "Zaporozhian_Cossacks", "Khmelnytsky_Uprising"],

    'Israel/Palestine' : 
    ["Nakba","Mandatory_Palestine","1948_Arab-Israeli_War","David_Ben-Gurion","Yasser_Arafat","Six-Day_War","Yom_Kippur_War","Hummus","Falafel","Shawarma","First_Intifada",
     "United_Nations_Partition_Plan_for_Palestine", "Intercommunal_conflict_in_Mandatory_Palestine", "Lehi_(militant_group)", "Irgun", "Ze'ev_Jabotinsky",
     "Haganah", "1947–1948_civil_war_in_Mandatory_Palestine", "1948_Arab–Israeli_War", "Yitzhak_Rabin", "Palmach", "Moshe_Dayan", "Jewish_exodus_from_the_Muslim_world", 
     "1936–1939_Arab_revolt_in_Palestine", "Amin_al-Husseini", "1948_Palestinian_expulsion_and_flight", "List_of_towns_and_villages_depopulated_during_the_1947–1949_Palestine_war", "Plan_Dalet", 
     "Abd_al-Qadir_al-Husayni", "1929_Hebron_massacre", "Causes_of_the_1948_Palestinian_expulsion_and_flight", "Deir_Yassin_massacre", "Menachem_Begin", "Kfar_Etzion_massacre", "Hebrew_language", 
     "Suez_Crisis", "Six-Day_War", "Egypt–Israel_peace_treaty", "Palestinian_Arabic", "Culture_of_Palestine", "Palestinian_cuisine", "Samih_al-Qasim", "Mahmoud_Darwish", "Origin_of_the_Palestinians"],
    
    'Italian_Fascism' : 
    ["Benito_Mussolini","Fall_of_the_Fascist_regime_in_Italy","Italian_Social_Republic","Rodolfo_Graziani","Ardeatine_massacre","Sant%27Anna_di_Stazzema_massacre","Istrian-Dalmatian_exodus","Foibe_massacres"],
    
    'Expulsion_Germans' : 
    ["Flight_and_expulsion_of_Germans_(1944–1950)","Expulsion_of_Germans_from_Czechoslovakia","Ústí_massacre","Sudetenland","Brno_death_march","Beneš_decrees",
     "Sudetendeutsche_Landsmannschaft","German_Expellees","Recovered_Territories","Flight_and_expulsion_of_Germans_from_Poland_during_and_after_World_War_II","Oder-Neisse_line"],
    
    'US_Civil_War' : 
    ["Ulysses_S._Grant","Sherman's_March_to_the_Sea","William_Tecumseh_Sherman","Union_Army","Confederate_States_Army","Robert_E._Lee","Joseph_E._Johnston","Alexander_H._Stephens","James_Longstreet",
     "United_Daughters_of_the_Confederacy","Army_of_Northern_Virginia","Jefferson_Davis","Origins_of_the_American_Civil_War","Confederate_States_of_America","Abraham_Lincoln","Battle_of_Gettysburg", 
     "Judah_P._Benjamin", "John_C._Breckinridge", "Joseph_Wheeler", "P._G._T._Beauregard", "Franklin_Buchanan", "Nathan_Bedford_Forrest", "Ku_Klux_Klan", "John_C._Frémont", "Joseph_Hooker", "George_Meade", 
     "Wilmington_massacre", "Red_Shirts_(United_States)", "United_Confederate_Veterans", "Confederate_History_Month", "Robert_E._Lee_Day", "Stonewall_Jackson", "Jim_Crow_laws", "John_Brown_(abolitionist)", 
     "William_Lloyd_Garrison", "Frederick_Douglass", "Thaddeus_Stevens", "Battle_of_the_Wilderness", "Battle_of_Antietam", "Reconstruction_era", "Emancipation_Proclamation", 
     "Thirteenth_Amendment_to_the_United_States_Constitution", "Slavery_in_the_United_States", "States'_rights", "Historiographic_issues_about_the_American_Civil_War"],
     
     'Slovak_Fascism':
     ["Slovak_National_Uprising","Jozef_Tiso","Slovak_Republic_(1939-1945)","Slovak_People%27s_Party","Ján_Vojtaššák","Alexander_Mach","Andrej_Hlinka"]
}

topics_df = pd.DataFrame([(key, value) for key in topics for value in topics[key]], columns=['Topic', 'Article'])

topics_df

,Topic,Article
0,Ukraine,Kyiv
1,Ukraine,Kievan_Rus'
2,Ukraine,Stepan_Bandera
3,Ukraine,Bohdan_Khmelnytsky
4,Ukraine,Cossacks
...,...,...
153,Slovak_Fascism,Slovak_Republic_(1939-1945)
154,Slovak_Fascism,Slovak_People%27s_Party
155,Slovak_Fascism,Ján_Vojtaššák
156,Slovak_Fascism,Alexander_Mach


In [85]:
reversion_correlations = correlations_topics_groups(wikiData_reverted[wikiData_reverted['Total_edits'] > 100], topics_df)


Correlations calculated
Topics assigned
Groups assigned


,Article1,Article2,Lang1,Lang2,Correlation,P_value,Topic_1,Topic_2,Group
582,Sherman's_March_to_the_Sea,Stonewall_Jackson,en,en,0.538177,1.803286e-13,US_Civil_War,US_Civil_War,Same Topic Same Language
148,Kyiv,Ukrainian_language,de,en,0.455909,1.221788e-09,Ukraine,Ukraine,Same Topic
644,Confederate_States_of_America,Stonewall_Jackson,en,en,0.443205,3.916547e-09,US_Civil_War,US_Civil_War,Same Topic Same Language
730,Stonewall_Jackson,William_Lloyd_Garrison,en,en,0.383919,4.970680e-07,US_Civil_War,US_Civil_War,Same Topic Same Language
407,Yom_Kippur_War,Rus'_people,ar,ru,0.378364,7.469993e-07,Israel/Palestine,Ukraine,Group 5
...,...,...,...,...,...,...,...,...,...
636,Confederate_States_of_America,Ukrainian_Soviet_Socialist_Republic,en,en,-0.171821,2.930132e-02,US_Civil_War,Ukraine,Same Language
262,Cossacks,Ukrainian_Soviet_Socialist_Republic,ru,en,-0.172198,2.894290e-02,Ukraine,Ukraine,Same Topic
622,Jefferson_Davis,Ukrainian_Soviet_Socialist_Republic,en,en,-0.190821,1.532110e-02,US_Civil_War,Ukraine,Same Language
298,Ukrainian_language,Nathan_Bedford_Forrest,en,en,-0.194930,1.321476e-02,Ukraine,US_Civil_War,Same Language


In [102]:
def summarise_correlations(df):

    print(df['Group'].value_counts(normalize=True).reset_index())
    print("\n #### \n")
    print(df[df['Correlation'] > 0.3]['Group'].value_counts(normalize=True).reset_index())
    

In [104]:
summarise_correlations(reversion_correlations)
reversion_correlations.head(10)


                      Group  proportion
0                   Group 5    0.312162
1             Same Language    0.305405
2  Same Topic Same Language    0.216216
3               Same Topic     0.154054
4              Same Article    0.012162

 #### 

                      Group  proportion
0  Same Topic Same Language    0.454545
1                   Group 5    0.272727
2               Same Topic     0.181818
3              Same Article    0.090909


,Article1,Article2,Lang1,Lang2,Correlation,P_value,Topic_1,Topic_2,Group
582,Sherman's_March_to_the_Sea,Stonewall_Jackson,en,en,0.538177,1.803286e-13,US_Civil_War,US_Civil_War,Same Topic Same Language
148,Kyiv,Ukrainian_language,de,en,0.455909,1.221788e-09,Ukraine,Ukraine,Same Topic
644,Confederate_States_of_America,Stonewall_Jackson,en,en,0.443205,3.916547e-09,US_Civil_War,US_Civil_War,Same Topic Same Language
730,Stonewall_Jackson,William_Lloyd_Garrison,en,en,0.383919,4.970680e-07,US_Civil_War,US_Civil_War,Same Topic Same Language
407,Yom_Kippur_War,Rus'_people,ar,ru,0.378364,7.469993e-07,Israel/Palestine,Ukraine,Group 5
140,Kyiv,Stonewall_Jackson,ru,en,0.360270,2.675101e-06,Ukraine,US_Civil_War,Group 5
142,Kyiv,Reconstruction_era,ru,en,0.360107,2.705023e-06,Ukraine,US_Civil_War,Group 5
572,Sherman's_March_to_the_Sea,Confederate_States_of_America,en,en,0.337677,1.183379e-05,US_Civil_War,US_Civil_War,Same Topic Same Language
166,Kyiv,Ukrainian_Soviet_Socialist_Republic,de,en,0.321582,3.187219e-05,Ukraine,Ukraine,Same Topic
75,Kyiv,Kyiv,en,de,0.312932,5.306168e-05,Ukraine,Ukraine,Same Article


In [105]:
edit_correlations = correlations_topics_groups(wikiData[wikiData['Total_edits'] > 1000], topics_df)
all_reversion_correlations = correlations_topics_groups(wikiData_reverted, topics_df)
all_edits_correlations = correlations_topics_groups(wikiData, topics_df)

Correlations calculated
Topics assigned
Groups assigned
Correlations calculated
Topics assigned
Groups assigned
Correlations calculated
Topics assigned
Groups assigned


In [106]:
summarise_correlations(edit_correlations)
edit_correlations.head(10)

                      Group  proportion
0                   Group 5    0.401600
1             Same Language    0.280000
2  Same Topic Same Language    0.167771
3               Same Topic     0.141486
4              Same Article    0.009143

 #### 

                      Group  proportion
0                   Group 5    0.410256
1  Same Topic Same Language    0.294872
2             Same Language    0.173077
3               Same Topic     0.096154
4              Same Article    0.025641


,Article1,Article2,Lang1,Lang2,Correlation,P_value,Topic_1,Topic_2,Group
4360,1948_Palestinian_expulsion_and_flight,1948_Palestinian_expulsion_and_flight,en,de,0.822070,9.866483e-41,Israel/Palestine,Israel/Palestine,Same Article
2486,Flight_and_expulsion_of_Germans_(1944–1950),Nikolai_Gogol,en,ru,0.688102,6.488655e-24,Expulsion_Germans,Ukraine,Group 5
3441,James_Longstreet,Massacres_of_Poles_in_Volhynia_and_Eastern_Gal...,en,ru,0.651947,7.485904e-21,US_Civil_War,Ukraine,Group 5
1514,Ukrainian_language,Amin_al-Husseini,uk,en,0.632325,2.334245e-19,Ukraine,Israel/Palestine,Group 5
2304,Yom_Kippur_War,Ivan_Franko,ar,uk,0.624938,8.004560e-19,Israel/Palestine,Ukraine,Group 5
3049,Ukrainian_Insurgent_Army,Organisation_of_Ukrainian_Nationalists,ru,ru,0.573937,1.737884e-15,Ukraine,Ukraine,Same Topic Same Language
3294,Union_Army,Confederate_States_Army,en,en,0.572785,2.036175e-15,US_Civil_War,US_Civil_War,Same Topic Same Language
3696,Abraham_Lincoln,Emancipation_Proclamation,de,en,0.559329,1.238308e-14,US_Civil_War,US_Civil_War,Same Topic
3227,Sherman's_March_to_the_Sea,Stonewall_Jackson,en,en,0.538177,1.803286e-13,US_Civil_War,US_Civil_War,Same Topic Same Language
1603,Holodomor,Yasser_Arafat,en,en,0.537565,1.943150e-13,Ukraine,Israel/Palestine,Same Language


In [107]:
summarise_correlations(all_reversion_correlations)
all_reversion_correlations.head(10)

                      Group  proportion
0                   Group 5    0.562728
1               Same Topic     0.194002
2             Same Language    0.151028
3  Same Topic Same Language    0.087181
4              Same Article    0.005061

 #### 

                      Group  proportion
0                   Group 5    0.460177
1  Same Topic Same Language    0.205752
2               Same Topic     0.175885
3             Same Language    0.137168
4              Same Article    0.021018


,Article1,Article2,Lang1,Lang2,Correlation,P_value,Topic_1,Topic_2,Group
32629,Oder–Neisse_line,Khreshchatyk,pl,en,0.978314,1.234888e-110,NaN,Ukraine,Group 5
32789,Oder–Neisse_line,Origin_of_the_Palestinians,pl,ar,0.968945,2.137425e-98,NaN,Israel/Palestine,Group 5
61974,Khreshchatyk,Origin_of_the_Palestinians,en,ar,0.964431,8.674403e-94,Ukraine,Israel/Palestine,Group 5
69853,Thaddeus_Stevens,Thirteenth_Amendment_to_the_United_States_Cons...,en,en,0.864516,2.280374e-49,US_Civil_War,US_Civil_War,Same Topic Same Language
53439,Syrniki,Rus'_people,de,en,0.826338,1.715568e-41,Ukraine,Ukraine,Same Topic
73749,1948_Palestinian_expulsion_and_flight,1948_Palestinian_expulsion_and_flight,en,de,0.822070,9.866483e-41,Israel/Palestine,Israel/Palestine,Same Article
1318,Fall_of_the_Fascist_regime_in_Italy,Battle_of_Gettysburg,en,de,0.799180,5.598350e-37,Italian_Fascism,US_Civil_War,Group 5
32611,Oder–Neisse_line,Ukrainian_War_of_Independence,pl,en,0.790618,1.069963e-35,NaN,Ukraine,Group 5
66444,Judah_P._Benjamin,P._G._T._Beauregard,en,de,0.789344,1.640534e-35,US_Civil_War,US_Civil_War,Same Topic
58941,Ukrainian_War_of_Independence,Origin_of_the_Palestinians,en,ar,0.771164,5.375144e-33,Ukraine,Israel/Palestine,Group 5


In [108]:
summarise_correlations(all_edits_correlations)
all_edits_correlations.head(10)

                      Group  proportion
0                   Group 5    0.570263
1               Same Topic     0.184803
2             Same Language    0.157052
3  Same Topic Same Language    0.082996
4              Same Article    0.004885

 #### 

                      Group  proportion
0                   Group 5    0.451859
1  Same Topic Same Language    0.198284
2               Same Topic     0.179218
3             Same Language    0.149666
4              Same Article    0.020972


,Article1,Article2,Lang1,Lang2,Correlation,P_value,Topic_1,Topic_2,Group
39310,Oder–Neisse_line,Khreshchatyk,pl,en,0.978314,1.234888e-110,NaN,Ukraine,Group 5
39486,Oder–Neisse_line,Origin_of_the_Palestinians,pl,ar,0.968945,2.137425e-98,NaN,Israel/Palestine,Group 5
73856,Khreshchatyk,Origin_of_the_Palestinians,en,ar,0.964431,8.674403e-94,Ukraine,Israel/Palestine,Group 5
83910,Thaddeus_Stevens,Thirteenth_Amendment_to_the_United_States_Cons...,en,en,0.864516,2.280374e-49,US_Civil_War,US_Civil_War,Same Topic Same Language
89212,Palestinian_cuisine,Palestinian_cuisine,en,de,0.863704,3.546263e-49,Israel/Palestine,Israel/Palestine,Same Article
84459,Battle_of_Antietam,List_of_towns_and_villages_depopulated_during_...,de,de,0.837371,1.480598e-43,US_Civil_War,Israel/Palestine,Same Language
64128,Syrniki,Rus'_people,de,en,0.826338,1.715568e-41,Ukraine,Ukraine,Same Topic
88081,1948_Palestinian_expulsion_and_flight,1948_Palestinian_expulsion_and_flight,en,de,0.822070,9.866483e-41,Israel/Palestine,Israel/Palestine,Same Article
48054,Taras_Shevchenko,Kobzar,de,de,0.815697,1.234258e-39,Ukraine,Ukraine,Same Topic Same Language
1370,Fall_of_the_Fascist_regime_in_Italy,Slovak_Republic_(1939-1945),en,en,0.806377,4.186542e-38,Italian_Fascism,Slovak_Fascism,Same Language
